In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
from datetime import date, timedelta

In [2]:
def find_sheet_name(sheet_names):
    iip_sheet_names = list(filter(lambda x: "spcn" in x.lower(), sheet_names))

    if len(iip_sheet_names) == 1:
        return iip_sheet_names[0]

    iip_sheet_names = list(filter(lambda x: "sp cn" in x.lower(), sheet_names))

    if len(iip_sheet_names) == 1:
        return iip_sheet_names[0]

    iip_sheet_names = list(
        filter(lambda x: "spcn" in x.lower() and "thang" in x.lower(), sheet_names)
    )

    if len(iip_sheet_names) == 1:
        return iip_sheet_names[0]
    
    iip_sheet_names = list(filter(lambda x: "sp" in x.lower(), sheet_names))

    if len(iip_sheet_names) == 1:
        return iip_sheet_names[0]

    # iip_sheet_names = list(filter(lambda x: "thang" in x.lower(), iip_sheet_names))

    # if len(iip_sheet_names) == 1:
    #     return iip_sheet_names[0]

    if len(sheet_names) == 43:
        return "9"

    # iip_sheet_names = list(filter(lambda x: "gtcn" in x.lower(), sheet_names))

    # if len(iip_sheet_names) == 1:
    #     return iip_sheet_names[0]

    raise RuntimeError(sheet_names, iip_sheet_names)

In [3]:
en_to_vn = {
    "Coal (pure)": "Than đá (than sạch)",
    "Extracted crude oil": "Dầu mỏ thô khai thác",
    "Natural gas (in the form of air)": "Khí đốt thiên nhiên dạng khí",
    "Liquidized gas (LPG)": "Khí hoá lỏng (LPG)",
    "Petroleum": "Xăng, dầu",
    "Aluminium": "Alumin",
    "Processed fishery products": "Thuỷ hải sản chế biến",
    "Fresh milk": "Sữa tươi",
    "Powder milk": "Sữa bột",
    "Refined sugar": "Đường kính",
    "Monosodium glutamate": "Bột ngọt",
    "Animal feed": "Thức ăn cho gia súc",
    "Aquatic feed": "Thức ăn cho thủy sản",
    "Beer": "Bia",
    "Cigarettes": "Thuốc lá điếu",
    "Textile fabric from natural yarn": "Vải dệt từ sợi tự nhiên",
    "Textile fabric from polyester or artificial yarn": "Vải dệt từ sợi tổng hợp hoặc sợi nhân tạo",
    "Clothes": "Quần áo mặc thường",
    "Leather footgear": "Giày, dép da",
    "Urea fertilizer": "Phân U rê",
    "N.P.K mixed fertilizer": "Phân hỗn hợp N.P.K",
    "Chemical paint": "Sơn hoá học ",
    "Cement": "Xi măng",
    "Crude steel, iron": "Sắt, thép thô",
    "Laminated steel": "Thép cán",
    "Steel bars and corners": "Thép thanh, thép góc",
    "Mobile phone": "Điện thoại di động",
    "Phone accessories": "Linh kiện điện thoại",
    "Television": "Tivi ",
    "Automobile": "Ô tô",
    "Motorbike": "Xe máy",
    "Generated electricity": "Điện sản xuất",
    "Running water": "Nước máy thương phẩm",
}

In [4]:
def get_spcn(xl, sheet_name, date):
    df = xl.parse(sheet_name)

    df = df.dropna(axis=1, how="all").dropna(axis=0, how="all")

    df[df.columns[0]] = (
        df[df.columns[0]]
        .replace(r"\s+", " ", regex=True)
        .replace(r"\n", " ", regex=True)
    )

    df[df.columns[0]] = df[df.columns[0]].str.strip()

    df.reset_index(inplace=True, drop=True)

    df = df.drop(
        list(
            range(
                df[df.columns[0]][
                    (df[df.columns[0]] == "Than đá (than sạch)")
                    | ((df[df.columns[0]] == "Coal (pure)"))
                ].index[0]
            )
        )
    )

    if len(df[df.columns[0]][df[df.columns[0]] == "Coal (pure)"].index) > 0:
        df[df.columns[0]] = df[df.columns[0]].map(en_to_vn)

    df = df.drop(columns=[df.columns[1]])

    if len(df.columns) == 5 and int(date.split("-")[1]) == 1:
        df.columns = ["Ngành", "Total-Pre-Month", "Total-M", "MOM", "YOY"]
        df["Total-YTD"] = df["Total-M"]
        df["YTD-YOY"] = df["YOY"]
    elif len(df.columns) == 6:
        df.columns = [
            "Ngành",
            "Total-Pre-Month",
            "Total-M",
            "Total-YTD",
            "YOY",
            "YTD-YOY",
        ]
    elif len(df.columns) == 8:
        df = df.drop(columns=[df.columns[6], df.columns[7]])
        df.columns = [
            "Ngành",
            "Total-Pre-Month",
            "Total-M",
            "Total-YTD",
            "YOY",
            "YTD-YOY",
        ]
    else:
        raise RuntimeError

    df["Total-Pre-Month"] = df["Total-Pre-Month"].astype(float)
    df["Total-M"] = df["Total-M"].astype(float)
    df["Total-YTD"] = df["Total-YTD"].astype(float)
    df["YOY"] = (df["YOY"] - 100).astype(float)
    df["YTD-YOY"] = (df["YTD-YOY"] - 100).astype(float)

    df = df[df["YOY"].notna()]
    df = df[df["YTD-YOY"].notna()]

    df["MOM"] = 100 * (df["Total-M"] / df["Total-Pre-Month"]) - 100

    df["Date"] = date
    df["Date"] = pd.to_datetime(df["Date"], dayfirst=False)

    df["Month"] = df["Date"].dt.strftime("%Y-%m")
    df["PreMonth"] = (
        df["Date"]
        .apply(
            lambda x: (
                x.replace(month=x.month - 1, day=1)
                if x.month > 1
                else x.replace(year=x.year - 1, month=12, day=1)
            )
        )
        .dt.strftime("%Y-%m")
    )

    # df["PreYearMonth"] = (
    #     df["Date"]
    #     .apply(lambda x: x.replace(year=x.year - 1, day=1))
    #     .dt.strftime("%Y-%m")
    # )

    return df


xl = pd.ExcelFile("../datas/2017-03-29-Bieu-3-2017-1.xlsx")

get_spcn(xl, find_sheet_name(xl.sheet_names), "2017-03-29")

,Ngành,Total-Pre-Month,Total-M,Total-YTD,YOY,YTD-YOY,MOM,Date,Month,PreMonth
4,Than đá (than sạch),5670.9,4019.2,9690.1,1.578896,-5.552874,-29.125888,2017-03-29,2017-03,2017-02
5,Dầu mỏ thô khai thác,2272.0,1168.0,3440.0,-14.117647,-14.851485,-48.591549,2017-03-29,2017-03,2017-02
6,Khí đốt thiên nhiên dạng khí,1515.0,1000.0,2515.0,1.926409,-8.913114,-33.993399,2017-03-29,2017-03,2017-02
7,Khí hoá lỏng (LPG),130.5,70.4,200.9,2.579533,-3.938969,-46.053640,2017-03-29,2017-03,2017-02
8,Thuỷ hải sản chế biến,375.4,430.1,805.5,15.000000,10.166908,14.571124,2017-03-29,2017-03,2017-02
9,Sữa tươi,189.5,104.2,293.7,2.054285,12.734145,-45.013193,2017-03-29,2017-03,2017-02
10,Sữa bột,16.5,9.3,25.8,15.486985,5.525086,-43.636364,2017-03-29,2017-03,2017-02
11,Đường kính,476.6,312.7,789.3,-6.200000,-7.367391,-34.389425,2017-03-29,2017-03,2017-02
12,Bột ngọt,45.3,23.8,69.1,3.133669,5.192788,-47.461369,2017-03-29,2017-03,2017-02
13,Thức ăn cho gia súc,1944.7,2071.2,4015.9,16.000000,9.346951,6.504859,2017-03-29,2017-03,2017-02


In [5]:
import os
import contextlib

dfs_spcn_origin = []
for file_name in sorted(
    os.listdir("../datas"),
    key=lambda filename: f"{filename.split('-')[0]}-{filename.split('-')[1]}-{filename.split('-')[2]}",
    reverse=True,
):
    if file_name.startswith(".~"):
        continue

    try:
        file_path = os.path.join("../datas", file_name)

        year = int(file_name.split("-")[0])
        month = int(file_name.split("-")[1])
        day = int(file_name.split("-")[2])

        if year < 2013:
            continue

        xl = pd.ExcelFile(file_path)

        df_spcn = get_spcn(xl, find_sheet_name(xl.sheet_names), f"{year}-{month}-{day}")

        dfs_spcn_origin.append(df_spcn)
    except Exception as e:
        print(file_name)
        raise e

In [6]:
dfs_spcn = pd.concat(dfs_spcn_origin, ignore_index=True, axis=0)
dfs_spcn["Date"] = pd.to_datetime(dfs_spcn["Date"], dayfirst=False)
dfs_spcn

,Ngành,Total-Pre-Month,Total-M,Total-YTD,YOY,YTD-YOY,MOM,Date,Month,PreMonth
0,Than đá (than sạch),3384.865683,3070.836840,33623.149255,-10.125086,-4.187632,-9.277439,2024-09-29,2024-09,2024-08
1,Dầu mỏ thô khai thác,684.200000,663.800000,6117.249000,2.066548,-5.907889,-2.981584,2024-09-29,2024-09,2024-08
2,Khí đốt thiên nhiên dạng khí,449.990000,393.466667,4807.946667,-18.462644,-16.529138,-12.561020,2024-09-29,2024-09,2024-08
3,Khí hoá lỏng (LPG),62.989977,58.056896,559.873783,-10.188120,-14.999432,-7.831533,2024-09-29,2024-09,2024-08
4,"Xăng, dầu",1625.894088,1575.385822,13528.158324,148.804080,20.308520,-3.106492,2024-09-29,2024-09,2024-08
...,...,...,...,...,...,...,...,...,...,...
4564,Xe chở khách,4.820754,4.913979,4.913979,-8.391633,-8.391633,1.933828,2013-01-29,2013-01,2012-12
4565,Xe tải,2.564023,2.511169,2.511169,38.241708,38.241708,-2.061373,2013-01-29,2013-01,2012-12
4566,Xe máy,360.037127,342.804343,342.804343,28.609839,28.609839,-4.786391,2013-01-29,2013-01,2012-12
4567,Điện sản xuất,9.869662,9.787661,9.787661,25.748727,25.748727,-0.830843,2013-01-29,2013-01,2012-12


In [7]:
convert_nganh = {
    "Than đá (than sạch)": "Than đá (than sạch)",
    "Dầu mỏ thô khai thác": "Dầu mỏ thô khai thác",
    "Khí đốt thiên nhiên dạng khí": "Khí đốt thiên nhiên dạng khí",
    "Khí hoá lỏng (LPG)": "Khí hoá lỏng (LPG)",
    "Xăng, dầu": "Xăng, dầu",
    "Alumin": "Alumin",
    "Thuỷ hải sản chế biến": "Thuỷ hải sản chế biến",
    "Sữa tươi": "Sữa tươi",
    "Sữa bột": "Sữa bột",
    "Đường kính": "Đường kính",
    "Bột ngọt": "Bột ngọt",
    "Thức ăn cho gia súc": "Thức ăn cho gia súc",
    "Thức ăn cho thủy sản": "Thức ăn cho thủy sản",
    "Bia": "Bia",
    "Thuốc lá điếu": "Thuốc lá điếu",
    "Vải dệt từ sợi tự nhiên": "Vải dệt từ sợi tự nhiên",
    "Vải dệt từ sợi tổng hợp hoặc sợi nhân tạo": "Vải dệt từ sợi tổng hợp hoặc sợi nhân tạo",
    "Quần áo mặc thường": "Quần áo mặc thường",
    "Giày, dép da": "Giày, dép da",
    "Phân U rê": "Phân U rê",
    "Phân hỗn hợp N.P.K": "Phân hỗn hợp N.P.K",
    "Sơn hoá học": "Sơn hoá học",
    "Xi măng": "Xi măng",
    "Sắt, thép thô": "Sắt, thép thô",
    "Thép cán": "Thép cán",
    "Thép thanh, thép góc": "Thép thanh, thép góc",
    "Điện thoại di động": "Điện thoại di động",
    "Linh kiện điện thoại": "Linh kiện điện thoại",
    "Tivi": "Tivi",
    "Ô tô": "Ô tô",
    "Xe máy": "Xe máy",
    "Điện sản xuất": "Điện sản xuất",
    "Nước máy thương phẩm": "Nước máy thương phẩm",
    "Ti vi": "Tivi",
    "Xăng, dầu các loại": "Xăng, dầu",
    "Bia các loại": "Bia",
    "Ti vi các loại": "Tivi",
    "Xăng dầu": "Xăng, dầu",
    "Tivi các loại": "Tivi",
    "Sơn hoá học ": "Sơn hoá học",
    "Tivi ": "Tivi",
    "Dầu gội đầu, dầu xả": "Dầu gội đầu, dầu xả",
    "Sữa tắm, sữa rửa mặt": "Sữa tắm, sữa rửa mặt",
    "Phân Ure": "Phân U rê",
    "Dầu mỏ thô khai thác trong nước": "Dầu mỏ thô khai thác",
    "Phân hỗn hợp (N, P, K)": "Phân hỗn hợp N.P.K",
    "Sơn hoá học các loại": "Sơn hoá học",
    "Bột giặt và các chế phẩm dùng để tẩy, rửa": "Bột giặt và các chế phẩm dùng để tẩy, rửa",
    "Gạch xây bằng đất nung": "Gạch xây bằng đất nung",
    "Xe chở khách": "Xe chở khách",
    "Xe tải": "Xe tải",
    "Giày, dép, ủng bằng da giả cho người lớn": "Giày, dép da",
}

for pl in dfs_spcn["Ngành"].unique().tolist():
    assert pl in convert_nganh.keys(), pl

dfs_spcn["Ngành"] = dfs_spcn["Ngành"].map(convert_nganh)
dfs_spcn

,Ngành,Total-Pre-Month,Total-M,Total-YTD,YOY,YTD-YOY,MOM,Date,Month,PreMonth
0,Than đá (than sạch),3384.865683,3070.836840,33623.149255,-10.125086,-4.187632,-9.277439,2024-09-29,2024-09,2024-08
1,Dầu mỏ thô khai thác,684.200000,663.800000,6117.249000,2.066548,-5.907889,-2.981584,2024-09-29,2024-09,2024-08
2,Khí đốt thiên nhiên dạng khí,449.990000,393.466667,4807.946667,-18.462644,-16.529138,-12.561020,2024-09-29,2024-09,2024-08
3,Khí hoá lỏng (LPG),62.989977,58.056896,559.873783,-10.188120,-14.999432,-7.831533,2024-09-29,2024-09,2024-08
4,"Xăng, dầu",1625.894088,1575.385822,13528.158324,148.804080,20.308520,-3.106492,2024-09-29,2024-09,2024-08
...,...,...,...,...,...,...,...,...,...,...
4564,Xe chở khách,4.820754,4.913979,4.913979,-8.391633,-8.391633,1.933828,2013-01-29,2013-01,2012-12
4565,Xe tải,2.564023,2.511169,2.511169,38.241708,38.241708,-2.061373,2013-01-29,2013-01,2012-12
4566,Xe máy,360.037127,342.804343,342.804343,28.609839,28.609839,-4.786391,2013-01-29,2013-01,2012-12
4567,Điện sản xuất,9.869662,9.787661,9.787661,25.748727,25.748727,-0.830843,2013-01-29,2013-01,2012-12


In [8]:
merge_df = pd.merge(
    left=dfs_spcn,
    right=dfs_spcn,
    left_on=["Ngành", "Month"],
    right_on=["Ngành", "PreMonth"],
    how="left",
)
merge_df["Merged_Total"] = np.where(
    merge_df["Total-Pre-Month_y"].isna(),
    merge_df["Total-M_x"],
    merge_df["Total-Pre-Month_y"],
)
merge_df = merge_df.drop(
    columns=[
        "Total-Pre-Month_y",
        "Total-M_y",
        "Total-YTD_y",
        "YOY_y",
        "YTD-YOY_y",
        "MOM_y",
        "Date_y",
        "Month_y",
        "PreMonth_y",
    ]
)
merge_df = merge_df.rename(
    columns={
        "Total-Pre-Month_x": "Total-Pre-Month",
        "Total-M_x": "E-Total-M",
        "Total-YTD_x": "Total-YTD",
        "YOY_x": "YOY",
        "YTD-YOY_x": "YTD-YOY",
        "MOM_x": "E-MOM",
        "Date_x": "Date",
        "Month_x": "Month",
        "PreMonth_x": "PreMonth",
        "Merged_Total": "Total-M",
    }
)
merge_df["MOM"] = 100 * (merge_df["Total-M"] / merge_df["Total-Pre-Month"]) - 100
merge_df[merge_df["Ngành"] == "Phân U rê"]

,Ngành,Total-Pre-Month,E-Total-M,Total-YTD,YOY,YTD-YOY,E-MOM,Date,Month,PreMonth,Total-M,MOM
19,Phân U rê,209.626660,211.439938,2025.499808,-2.356523,7.976702,0.865003,2024-09-29,2024-09,2024-08,211.439938,0.865003
52,Phân U rê,226.048156,202.350000,1806.783725,16.131752,8.886694,-10.483676,2024-08-29,2024-08,2024-07,209.626660,-7.264601
85,Phân U rê,229.848484,207.563028,1585.948083,-15.144017,6.791923,-9.695716,2024-07-29,2024-07,2024-06,226.048156,-1.653406
118,Phân U rê,208.717415,214.369028,1362.905599,-6.620311,9.869549,2.707782,2024-06-29,2024-06,2024-05,229.848484,10.124248
151,Phân U rê,232.370336,218.853092,1158.672247,-10.070897,14.616857,-5.817113,2024-05-29,2024-05,2024-04,208.717415,-10.178976
...,...,...,...,...,...,...,...,...,...,...,...,...
4424,Phân U rê,677.600000,177.700000,855.300000,12.200000,58.700000,-73.775089,2013-05-29,2013-05,2013-04,856.184400,26.355431
4456,Phân U rê,527.500000,169.700000,697.200000,55.600000,83.300000,-67.829384,2013-04-29,2013-04,2013-03,677.600000,28.454976
4488,Phân U rê,354.500000,173.900000,528.400000,91.500000,94.800000,-50.944993,2013-03-29,2013-03,2013-02,527.500000,48.801128
4520,Phân U rê,170.064700,164.130000,334.194700,77.060555,85.222542,-3.489672,2013-02-28,2013-02,2013-01,354.500000,108.450078


In [9]:
dfs_spcn = merge_df

In [10]:
dfs_spcn.to_csv("../csv/spcn.csv")

In [11]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

df_show = dfs_spcn[
    (
        (dfs_spcn["Ngành"] == "Thuỷ hải sản chế biến")
        # | (dfs_spcn["Ngành"] == "Sữa tươi")
        # | (dfs_spcn["Ngành"] == "Sữa bột")
        # | (dfs_spcn["Ngành"] == "Đường kính")
        # | (dfs_spcn["Ngành"] == "Quần áo mặc thường")
        # | (dfs_spcn["Ngành"] == "Giày, dép da")
        | (dfs_spcn["Ngành"] == "Phân U rê")
        | (dfs_spcn["Ngành"] == "Phân hỗn hợp N.P.K")
        # | (dfs_spcn["Ngành"] == "Sắt, thép thô")
        # | (dfs_spcn["Ngành"] == "Thép cán")
        # | (dfs_spcn["Ngành"] == "Thép thanh, thép góc")
        # | (dfs_spcn["Ngành"] == "Điện thoại di động")
        # | (dfs_spcn["Ngành"] == "Linh kiện điện thoại")
        # | (dfs_spcn["Ngành"] == "Điện sản xuất")
    )
    & (dfs_spcn["Date"] > "2019-01-01")
]

fig = px.area(df_show, x="Date", y="YOY", color="Ngành")

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=[
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(count=3, label="3y", step="year", stepmode="backward"),
            dict(count=5, label="5y", step="year", stepmode="backward"),
            dict(count=7, label="7y", step="year", stepmode="backward"),
            dict(count=10, label="10y", step="year", stepmode="backward"),
            dict(step="all"),
        ]
    ),
)
# fig.update_layout(yaxis_range=[-100,600])

fig.show()